# I. Scrape Gay/Lesbian List from Box Office Mojo 

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import requests

gay_list = []
url_list = []
gay_movies_urls = []
    
for i in range(1,5):
    url = 'http://www.boxofficemojo.com/genres/chart/?view=main&sort=gross&order=DESC&pagenum=' + str(i) + '&id=gay.htm'
    url_list.append(url)

for url in url_list:
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,'lxml')

    tables = soup.find_all("table")
    rows = [row for row in tables[2].find_all('tr')]
    rows = rows[2:102]

    for row in rows:
        cells = row.find_all('td')
        for i in range(len(cells)//8):
            gay_list.append([a.text for a in cells[i*8:(i+1)*8]])
            gay_movies_urls.append(row.find('a')['href'])
            
df_gay_list = pd.DataFrame(gay_list)

list_head = ['rank', 'title', 'distributor','life_gross', 'life_theaters', 'opening_gross', 'opening_theaters', 'list_date'] 

df_gay_list.columns = list_head
df_gay_list = df_gay_list.drop(['distributor','life_gross','list_date'], 1)
df_gay_list.head()


,rank,title,life_theaters,opening_gross,opening_theaters
0,1,The Birdcage,"2,285","$18,275,828","1,950"
1,2,Interview with the Vampire,"2,604","$36,389,705","2,604"
2,3,The Imitation Game,"2,402","$479,352",4
3,4,Brokeback Mountain,"2,089","$547,425",5
4,5,The Talented Mr. Ripley,"2,369","$12,738,237","2,307"


In [3]:
df_gay_list.shape

(341, 5)

# II. Scrape info/variables from each movie in list 

In [5]:
mojo_url = 'http://www.boxofficemojo.com' 

# Function to aquire upper right box info
def get_right_value(soup, field_name):
    obj = soup.find(text = re.compile(field_name))
    if not obj:
        return None
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text
    else:
        return None
    
gay_movies_info = []

# Loop through all movies in list and scrape variables
for url in gay_movies_urls:
    url = mojo_url + url

    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,'lxml')
    
    title_string = soup.find('title').text
    title_string.split('(')
    title = title_string.split('(')[0].strip()
    
    dtg = get_right_value(soup, 'Domestic Total')
    distributor = get_right_value(soup, 'Distributor')
    release = get_right_value(soup, 'Release Date')
    genre = get_right_value(soup, 'Genre:')
    runtime = get_right_value(soup, 'Runtime')
    rating = get_right_value(soup, 'MPAA')
    budget = get_right_value(soup, 'Production Budget')

    box_tab = soup.find_all(class_='mp_box_tab')
    box = soup.find_all(class_='mp_box_content')

    foreign = "NaN"
    rows = "NaN"
    director = None
    actor = None
    actors = None
    producer = None
    composer = None
    writer = None
    oscar = None
    number_genres = "NaN"
    number_charts = "NaN"

    for i in range(0,len(box_tab)):
        
        if box_tab[i].text == 'Total Lifetime Grosses':
            table = box[i].find('table')
            rows = [row for row in table.find_all('tr')]
                                           
            for r in range(0,len(rows)):
                if rows[r].find('a'):
                    foreign = rows[r].find_all('td')[1].text
                            
        if box_tab[i].text == 'The Players': 
            table = box[i].find('table')
            rows=[row for row in table.find_all('tr')]
            
            for r in range(0,len(rows)):
                try: 
                    if rows[r].find('a').text == 'Director:' or rows[r].find('a').text == 'Directors:':
                        director_line = rows[r].find_all('td')
                        director = director_line[1].text
                except:
                    pass

                try:
                    if rows[r].find('a').text == 'Actor:' or rows[r].find('a').text == 'Actors:':
                        actor_line = rows[r].find_all('td')
                        actor = actor_line[1].text                     
                except:
                    pass
        
                try:
                    if rows[r].find('a').text == 'Producer:' or rows[r].find('a').text == 'Producers:':
                        producer_line = rows[r].find_all('td')
                        producer = producer_line[1].text
                except:
                    pass
        
                try:
                    if rows[r].find('a').text == 'Composer:' or rows[r].find('a').text == 'Composers:':
                        composer_line = rows[r].find_all('td')
                        composer = composer_line[1].text
                except:
                    pass

                try:
                    if rows[r].find('a').text == 'Writer:' or rows[r].find('a').text == 'Writers:':
                        writer_line = rows[r].find_all('td')
                        writer = writer_line[1].text
                except:
                    pass    
     
        try:
            if box_tab[i].text.split(' ')[0] == release.split(', ')[1]:
                table = box[i].find('table')
                rows=[row for row in table.find_all('tr')]
                cells = rows[0].find_all('b')
                oscar = cells[0].text 
        except:
            pass  

        if box_tab[i].text == 'Genres':
            table = box[i].find('table')
            rows = [row for row in table.find_all('tr')]
            number_genres = len(rows) - 1
        
        if box_tab[i].text == 'Charts':
            table = box[i].find('table')
            rows = [row for row in table.find_all('tr')]
            number_charts = len(rows) - 1
 
        
    one_movie_info = [title, dtg, distributor, release, genre, runtime, rating, budget, foreign, \
            director, actor, producer, composer, writer, oscar, number_genres, number_charts]

    gay_movies_info.append(one_movie_info)
    
print gay_movies_info[:10]

[[u'The Birdcage', u'$124,060,553', u'MGM', u'March 8, 1996', u'Comedy', u'1 hrs. 57 min.', u'R', u'N/A', u'\xa0$61,200,000', u'Mike Nichols', u'Robin Williams', u'Mike Nichols', None, None, u'Nominated for One Oscar.', 3, 46], [u'Interview with the Vampire', u'$105,264,608', u'Warner Bros.', u'November 11, 1994', u'Period Horror', u'2 hrs. 3 min.', u'R', u'$60 million', u'\xa0$118,400,000', u'Neil Jordan', u'Tom CruiseBrad PittKirsten DunstAntonio BanderasThandie Newton*', None, u'Elliot Goldenthal', None, u'Nominated for Two Oscars.', 3, 35], [u'The Imitation Game', u'$91,125,683', u'Weinstein Company', u'November 28, 2014', u'Drama', u'1 hrs. 54 min.', u'PG-13', u'N/A', u'\xa0$142,430,025', u'Morten Tyldum', u'Benedict CumberbatchKeira KnightleyMatthew GoodeCharles DanceMark Strong', u'Nora GrossmanGraham Moore (executive)Ido OstrowskyTeddy Schwarzman', u'Alexandre Desplat', u'Graham Moore', u'Nominated for Eight Oscars, Including One Win.', 2, 25], [u'Brokeback Mountain', u'$83,043

# III. Crate dataframe

In [9]:
# Create dataframe
df_movies = pd.DataFrame(gay_movies_info, columns = ['title', 'dtg', 'distributor','release', 'genre',\
                                                     'runtime', 'rating', 'budget','foreign', 'director', \
                                                     'actor', 'producer', 'composer', 'writer', 'oscar',\
                                                     'number_genres', 'number_charts']) 

df_movies.head()

,title,dtg,distributor,release,genre,runtime,rating,budget,foreign,director,actor,producer,composer,writer,oscar,number_genres,number_charts
0,The Birdcage,"$124,060,553",MGM,"March 8, 1996",Comedy,1 hrs. 57 min.,R,N/A,"$61,200,000",Mike Nichols,Robin Williams,Mike Nichols,None,None,Nominated for One Oscar.,3,46
1,Interview with the Vampire,"$105,264,608",Warner Bros.,"November 11, 1994",Period Horror,2 hrs. 3 min.,R,$60 million,"$118,400,000",Neil Jordan,Tom CruiseBrad PittKirsten DunstAntonio Bander...,None,Elliot Goldenthal,None,Nominated for Two Oscars.,3,35
2,The Imitation Game,"$91,125,683",Weinstein Company,"November 28, 2014",Drama,1 hrs. 54 min.,PG-13,N/A,"$142,430,025",Morten Tyldum,Benedict CumberbatchKeira KnightleyMatthew Goo...,Nora GrossmanGraham Moore (executive)Ido Ostro...,Alexandre Desplat,Graham Moore,"Nominated for Eight Oscars, Including One Win.",2,25
3,Brokeback Mountain,"$83,043,761",Focus Features,"December 9, 2005",Western,2 hrs. 15 min.,R,$14 million,"$95,018,998",Ang Lee,Heath LedgerJake GyllenhaalAnne HathawayMichel...,None,None,None,"Nominated for Eight Oscars, Including Three Wins.",4,25
4,The Talented Mr. Ripley,"$81,298,265",Paramount,"December 25, 1999",Thriller,2 hrs. 19 min.,R,$40 million,"$47,500,000",Anthony Minghella,Matt DamonGwyneth PaltrowJude LawCate Blanchet...,Sydney Pollack (executive),Gabriel Yared,None,Nominated for Five Oscars.,2,20


In [10]:
df_movies.shape

(341, 17)

In [32]:
# Merge both dataframes and drop duplicates
df = pd.merge(df_movies, df_gay_list, on='title', how='outer')
df.drop(df.index[57:60], inplace = True)
df.reset_index(inplace=True, drop=True)
df.shape

(351, 21)

In [39]:
# Pickle dataframe
df.to_pickle('L1 - Dataframe1.pkl')

# IV. Clean data

In [42]:
df.dtypes

title                object
dtg                  object
distributor          object
release              object
genre                object
runtime              object
rating               object
budget               object
foreign              object
director             object
actor                object
producer             object
composer             object
writer               object
oscar                object
number_genres       float64
number_charts        object
rank                 object
life_theaters        object
opening_gross        object
opening_theaters     object
dtype: object

In [40]:
# Functions to clean data
import dateutil.parser

def object_to_int(string):
    try:
        return int(string)
    except:
        pass

def money_to_int(moneystring):
    try:
        moneystring =  moneystring.replace('$','').replace(',','')
        return int(moneystring)
    except:
        pass
    
def to_date(datestring):
    try:
        date = dateutil.parser.parse(datestring)
        return date
    except:
        pass
    
def runtime_to_minutes(runtimestring):
    try:
        runtime = runtimestring.split()
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None   

    
numberdict = {'None':0, 'One':1,'Two':2,'Three':3,'Four':4,
           'Five':5,'Six':6,'Seven':7,'Eight':8, 'Nine':9}
    
def sentence_to_number(oscarstring):
    try:
        if oscarstring ==  None:
            return int("0")
        else:
            oscar_list = oscarstring.split(' ')
            number = oscar_list[2]
            return int(numberdict[number])
    except:
        pass
    

In [43]:
df.dtg = df.dtg.apply(lambda x: money_to_int(x))
df.foreign = df.foreign.apply(lambda x: money_to_int(x))
df.budget = df.budget.apply(lambda x: money_to_int(x))
df.opening_gross = df.opening_gross.apply(lambda x: money_to_int(x))

df.number_charts = df.number_charts.apply(lambda x: object_to_int(x))
df.life_theaters = df.life_theaters.apply(lambda x: object_to_int(x))              
df.opening_theaters = df.opening_theaters.apply(lambda x: object_to_int(x))

df.release = df.release.apply(lambda x: to_date(x))
df['year'] = df.release.apply(lambda x: x.year)

df.runtime = df.runtime.apply(lambda x: runtime_to_minutes(x))

df.oscar = df.oscar.apply(lambda x: sentence_to_number(x))

df.actor = df.actor.apply(lambda x: re.findall(r'[A-Z][^A-Z]*', str(x)))
df.actor = df.actor.apply(lambda x: [i+j for i,j in zip(x[::2], x[1::2])])
df.actor = df.actor.apply(lambda x: len(x))

In [44]:
df.dtypes

title                       object
dtg                        float64
distributor                 object
release             datetime64[ns]
genre                       object
runtime                    float64
rating                      object
budget                     float64
foreign                    float64
director                    object
actor                        int64
producer                    object
composer                    object
writer                      object
oscar                      float64
number_genres              float64
number_charts              float64
rank                        object
life_theaters              float64
opening_gross              float64
opening_theaters           float64
year                       float64
dtype: object

In [45]:
# Create new variables 

def dummy_it(x):
    if str(x).strip() != "None":
        return "1"
    else:
        return "0"
    
df['director_dummy'] = df.director.apply(lambda x: dummy_it(x))
df['writer_dummy'] = df.writer.apply(lambda x: dummy_it(x))
df['composer_dummy'] = df.composer.apply(lambda x: dummy_it(x))
df['producer_dummy'] = df.producer.apply(lambda x: dummy_it(x))

In [47]:
df.shape

(351, 26)

In [50]:
import csv
df.to_csv('L2 - Dataframe1.csv')

In [49]:
df.head()

,title,dtg,distributor,release,genre,runtime,rating,budget,foreign,director,...,number_charts,rank,life_theaters,opening_gross,opening_theaters,year,director_dummy,writer_dummy,composer_dummy,producer_dummy
0,The Birdcage,124060553.0,MGM,1996-03-08,Comedy,117.0,R,NaN,61200000.0,Mike Nichols,...,46.0,1,NaN,18275828.0,NaN,1996.0,1,0,0,1
1,Interview with the Vampire,105264608.0,Warner Bros.,1994-11-11,Period Horror,123.0,R,NaN,118400000.0,Neil Jordan,...,35.0,2,NaN,36389705.0,NaN,1994.0,1,0,1,0
2,The Imitation Game,91125683.0,Weinstein Company,2014-11-28,Drama,114.0,PG-13,NaN,142430025.0,Morten Tyldum,...,25.0,3,NaN,479352.0,4.0,2014.0,1,1,1,1
3,Brokeback Mountain,83043761.0,Focus Features,2005-12-09,Western,135.0,R,NaN,95018998.0,Ang Lee,...,25.0,4,NaN,547425.0,5.0,2005.0,1,0,0,0
4,The Talented Mr. Ripley,81298265.0,Paramount,1999-12-25,Thriller,139.0,R,NaN,47500000.0,Anthony Minghella,...,20.0,5,NaN,12738237.0,NaN,1999.0,1,0,1,1
